# Thomas Ngo
# Tevin Vu

# 1. Use from dataset import * to load the module, then examine TRAINING_SET, TEST_SET, and MESSAGE.

In [1]:
import numpy as np
import copy
import sys
import string

from dataset import *
from numpy.random import default_rng

In [2]:
print(TRAINING_SET[0])

([0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], 'A')


In [3]:
print(TEST_SET[0])

([0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], 'A')


In [4]:
print(MESSAGE[0])

[1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


# 2. In order to use the images in TRAINING_SET, TEST_SET, and MESSAGE, convert them into two-dimensional NumPy arrays of feature vectors.

In [5]:
def convert_to_2d(dataset):
    copyDataset = copy.deepcopy(dataset)
    if len(copyDataset) == 0: return None
    res = []
    if len(copyDataset[0]) == 2:
        for x, _ in(copyDataset):
            x.insert(0,1)            # include bias
            res.append(np.array(x))
    else:
        for x in(copyDataset):
            x.insert(0,1)            # include bias
            res.append(np.array(x))
    
    return np.array(res)

In [6]:
TRAINING_SET_2D = convert_to_2d(TRAINING_SET)
TEST_SET_2D = convert_to_2d(TEST_SET)
MESSAGE_2D = convert_to_2d(MESSAGE)

In [7]:
TRAINING_SET_2D.shape

(52, 36)

In [8]:
TEST_SET_2D.shape

(26, 36)

In [9]:
MESSAGE_2D.shape

(37, 36)

In [10]:
TRAINING_SET_2D

array([[1, 0, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])

# 3. Implementing a Python function show(image) to reshape a one-dimensional feature vector into a 7×5 array, then print() the image. Test this function on some of the images in the training set.

In [11]:
def show(img):
    reshaped_img = np.reshape(np.array(img), (7,5))
#     print(reshaped_img)
    rows = len(reshaped_img)
    cols = len(reshaped_img[0])
    for row in range(rows):
        for col in range(cols):
            if reshaped_img[row][col] == 1:
                print('#', end='')
            else:
                print(' ', end='')
        print('\n', end='')

In [12]:
show(TRAINING_SET[0][0])

 ### 
#   #
#   #
#   #
#####
#   #
#   #


# 4.In order to use the character labels in TRAINING_SET and TEST_SET, convert them into an integer class vector using ord(), then use NumPy to convert them into 26 one-hot encoded vectors.

In [13]:
# adapted from https://www.kite.com/python/answers/how-to-do-one-hot-encoding-with-numpy-in-python

def convert_labels_to_int(dataset):
    res = []
    for _, letter in dataset:
        res.append(ord(letter)-ord('A'))
    tempRes = np.array(res)
    print(tempRes)
    shape = (tempRes.size, tempRes.max() + 1)
    print(shape)
    one_hot = np.zeros(shape)
    rows = np.arange(tempRes.size)
    print(rows)
    one_hot[rows, tempRes] = 1 # using fancy index
    return one_hot

In [14]:
TRAINING_SET_LABELS = convert_labels_to_int(TRAINING_SET)
print(TRAINING_SET_LABELS.shape)
TRAINING_SET_LABELS

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 22 23 24 25]
(52, 26)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]
(52, 26)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

# 5. Implement a multiclass perceptron as described in Section 2.3.1 of the textbook to recognize the letters A-Z.
## Begin with weights initialized to random values and a learning rate of 𝛼 = 0.01. Predict the letter by choosing the class with the largest W̅r∙X̅i value.
##  If the prediction is incorrect, train the multiclass perceptron by applying the update rule of Equation 2.24 in the textbook. Continue training until all items in the training set are correctly classified.


In [16]:
# use while-loop and no epochs
def check_for_misclassification(epochs, W, X_train):
    N = len(X_train)
    misclassification = 0
    pred_idx = 0
    for i in range(N):
        if pred_idx == 26:
            pred_idx = 0
        ascii_idx = np.argmax(np.matmul(W, X_train[i]))
        y_pred = chr(ascii_idx + ord('A'))
        if y_pred != string.ascii_uppercase[pred_idx]:
            misclassification += 1
        pred_idx += 1
    print(f'Epoch {epochs} - Accuracy: {N - misclassification}/{N}')
    return misclassification
    
    
def train(data_train, target_train):
    ALPHA = 0.01
    
    D = len(data_train[0])             # dimensions (including bias)
    N = len(data_train)                # training instance
    P = len(string.ascii_uppercase)    # number of perceptrons
    
    rng = default_rng()
    W = rng.uniform(-1, +1, (P, D))
    X_train = data_train
    Y_train = target_train
    
    idx = np.arange(N)
    w_idx = 0
    misclassification = 0
    epochs = 0
    while True:
        for i in range(N):
            if w_idx == P:
                w_idx = 0
            Y_hat = np.dot(W, X_train[i])
            y_pred_idx = np.argmax(Y_hat)

            # prediction is incorrect, use update rule of equation 2.24 in textbook
            if Y_train[i][y_pred_idx] == 0:
                W[w_idx] = W[w_idx] + ALPHA * X_train[i]
                W[y_pred_idx] = W[y_pred_idx] + (-ALPHA * X_train[i])
            w_idx += 1
            
        # check if all items in the training set are correctly classified
        misclassification = check_for_misclassification(epochs, W, X_train)
        if misclassification == 0:
            print('All items are correctly classified')
            print(f'number of epochs: {epochs}')
            return W
        else:
            misclassification = 0
        epochs += 1
    return W 

In [17]:
# ask professor why epoch 0 has poor accuracy while his binary perceptron has relatively high accuracy
# train model with TRAINING SET
mul_pct_model = train(TRAINING_SET_2D, TRAINING_SET_LABELS)
print(mul_pct_model.shape)
mul_pct_model

Epoch 0 - Accuracy: 4/52
Epoch 1 - Accuracy: 3/52
Epoch 2 - Accuracy: 3/52
Epoch 3 - Accuracy: 3/52
Epoch 4 - Accuracy: 4/52
Epoch 5 - Accuracy: 5/52
Epoch 6 - Accuracy: 6/52
Epoch 7 - Accuracy: 7/52
Epoch 8 - Accuracy: 8/52
Epoch 9 - Accuracy: 9/52
Epoch 10 - Accuracy: 9/52
Epoch 11 - Accuracy: 11/52
Epoch 12 - Accuracy: 15/52
Epoch 13 - Accuracy: 16/52
Epoch 14 - Accuracy: 17/52
Epoch 15 - Accuracy: 21/52
Epoch 16 - Accuracy: 20/52
Epoch 17 - Accuracy: 24/52
Epoch 18 - Accuracy: 24/52
Epoch 19 - Accuracy: 19/52
Epoch 20 - Accuracy: 24/52
Epoch 21 - Accuracy: 23/52
Epoch 22 - Accuracy: 26/52
Epoch 23 - Accuracy: 27/52
Epoch 24 - Accuracy: 30/52
Epoch 25 - Accuracy: 29/52
Epoch 26 - Accuracy: 36/52
Epoch 27 - Accuracy: 38/52
Epoch 28 - Accuracy: 41/52
Epoch 29 - Accuracy: 38/52
Epoch 30 - Accuracy: 40/52
Epoch 31 - Accuracy: 43/52
Epoch 32 - Accuracy: 42/52
Epoch 33 - Accuracy: 45/52
Epoch 34 - Accuracy: 45/52
Epoch 35 - Accuracy: 44/52
Epoch 36 - Accuracy: 42/52
Epoch 37 - Accuracy: 4

array([[-4.95999572e-01, -6.90368687e-01,  3.83840079e-01,
        -2.12482219e-01,  2.93956784e-01, -4.99503503e-01,
         7.38728292e-01,  4.33950422e-01, -6.56363242e-01,
        -2.73963186e-01,  4.35544119e-01, -2.07062657e-01,
         2.18344734e-01, -2.07456454e-01,  6.57367554e-01,
         3.42926702e-01, -2.31676680e-01, -2.39311445e-01,
         4.85474870e-01,  4.73257049e-01, -2.63514382e-01,
        -1.06024499e-01,  5.98318671e-01,  1.03115915e+00,
         6.14131858e-01,  1.08335074e+00, -6.34051603e-01,
        -2.83243155e-01, -7.23172527e-01, -9.44674095e-01,
        -4.85443178e-01, -2.63726897e-01,  2.98735762e-01,
        -8.98441924e-01, -2.12707402e-01,  1.08298973e+00],
       [-7.65452257e-01,  9.39356768e-01, -2.07981979e-01,
         8.23802715e-01, -1.81218973e-01, -6.99838891e-01,
         4.54862281e-01, -1.93400449e-01, -6.32940088e-01,
        -2.49623022e-01,  5.42241305e-01, -8.00894988e-01,
         3.56942968e-01,  3.91772613e-01, -2.65581155e-

# 6. Use your trained multiclass perceptron to classify each image in the test set.
## Apply the weights to all feature vectors in the test set at once using np.matmul(), then make predictions using np.amax().
## What accuracy do you obtain? If the accuracy is less than 100%, which test images are misclassified?


In [18]:
def test_model(dataset_2D, model):
    len_train_set = len(dataset_2D)
    w_idx = 0
    misclassification = 0
    for i in range(len_train_set):
        if w_idx == 26:
            w_idx = 0
        ascii_idx = np.argmax(np.dot(model, dataset_2D[i]))
        y_pred = chr(ascii_idx + ord('A'))
        print(f'Prediction: {y_pred}, Target: {string.ascii_uppercase[w_idx]}, {y_pred == string.ascii_uppercase[w_idx]}')
        if y_pred != string.ascii_uppercase[w_idx]:
            misclassification += 1
        w_idx += 1
    
    correct = len_train_set - misclassification
    res = correct if correct > 0 else 0
    print(f'\nAccuracy: {res}/{len_train_set} Percentage: {(res/len_train_set * 100):.2f}%')

In [19]:
test_model(TRAINING_SET_2D, mul_pct_model)

Prediction: A, Target: A, True
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Prediction: G, Target: G, True
Prediction: H, Target: H, True
Prediction: I, Target: I, True
Prediction: J, Target: J, True
Prediction: K, Target: K, True
Prediction: L, Target: L, True
Prediction: M, Target: M, True
Prediction: N, Target: N, True
Prediction: O, Target: O, True
Prediction: P, Target: P, True
Prediction: Q, Target: Q, True
Prediction: R, Target: R, True
Prediction: S, Target: S, True
Prediction: T, Target: T, True
Prediction: U, Target: U, True
Prediction: V, Target: V, True
Prediction: W, Target: W, True
Prediction: X, Target: X, True
Prediction: Y, Target: Y, True
Prediction: Z, Target: Z, True
Prediction: A, Target: A, True
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Predicti

In [20]:
test_model(TEST_SET_2D, mul_pct_model)

Prediction: E, Target: A, False
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Prediction: G, Target: G, True
Prediction: H, Target: H, True
Prediction: I, Target: I, True
Prediction: J, Target: J, True
Prediction: K, Target: K, True
Prediction: L, Target: L, True
Prediction: M, Target: M, True
Prediction: N, Target: N, True
Prediction: O, Target: O, True
Prediction: P, Target: P, True
Prediction: Q, Target: Q, True
Prediction: R, Target: R, True
Prediction: S, Target: S, True
Prediction: S, Target: T, False
Prediction: U, Target: U, True
Prediction: V, Target: V, True
Prediction: O, Target: W, False
Prediction: X, Target: X, True
Prediction: Y, Target: Y, True
Prediction: X, Target: Z, False

Accuracy: 22/26 Percentage: 84.62%


# 7. Implement and train a multinomial logistic regression classifier as described in Section 2.3.3 of the textbook. 
## Begin with weights initialized to random values and a learning rate of 𝛼 = 0.01. Predict the letter using the softmax activation function of Equation 2.29 in the textbook.
## Train the classifier by applying the update rule of Equation 2.37 in the textbook until all items in the training set are correctly classified.


In [22]:
# use while-loop and no epochs
def check_for_misclassification(epochs, W, X_train):
    N = len(X_train)
    misclassification = 0
    pred_idx = 0
    for i in range(N):
        if pred_idx == 26:
            pred_idx = 0
        ascii_idx = np.argmax(np.matmul(W, X_train[i]))
        y_pred = chr(ascii_idx + ord('A'))
        if y_pred != string.ascii_uppercase[pred_idx]:
            misclassification += 1
        pred_idx += 1
    print(f'Epoch {epochs} - Accuracy: {N - misclassification}/{N}')
    return misclassification
    
    
def mul_log_reg_train(data_train, target_train):
    ALPHA = 0.01
    LAMBDA = 0.60125 # penalty
    
    D = len(data_train[0])             # dimensions (including bias)
    N = len(data_train)                # training instance
    P = len(string.ascii_uppercase)    # number of perceptrons
    
    rng = default_rng()
    W = rng.uniform(-1, +1, (P, D))
    X_train = data_train
    Y_train = target_train
    
    idx = np.arange(N)
    w_idx = 0
    misclassification = 0
    epochs = 0
    while True:
        for i in range(N):
            if w_idx == P:
                w_idx = 0
                
            # Predict the letter using the softmax activation function of Equation 2.29 in the textbook
            Y_hat = np.dot(W, X_train[i])
            one_class_exp = np.exp(Y_hat)
            sum_classes_exp = np.sum(one_class_exp)
            probability = one_class_exp / sum_classes_exp
            y_pred_idx = np.argmax(probability)

            # prediction is incorrect, use update rule of equation 2.37 in textbook
            if Y_train[i][y_pred_idx] == 0:
                temp = W[w_idx]
                W = W * (1.0 - ALPHA * LAMBDA) + (-ALPHA * probability[w_idx] * X_train[i])
                W[w_idx] = temp * (1.0 - ALPHA * LAMBDA) + (ALPHA * (1.0 - probability[w_idx]) * X_train[i])
                
            w_idx += 1
            
        # check if all items in the training set are correctly classified
        misclassification = check_for_misclassification(epochs, W, X_train)
        if misclassification == 0:
            print('All items are correctly classified')
            print(f'number of epochs: {epochs}')
            return W
        else:
            misclassification = 0
        epochs += 1
    return W 

In [23]:
mul_log_reg_model = mul_log_reg_train(TRAINING_SET_2D, TRAINING_SET_LABELS)
print(mul_log_reg_model.shape)
mul_log_reg_model

Epoch 0 - Accuracy: 4/52
Epoch 1 - Accuracy: 4/52
Epoch 2 - Accuracy: 4/52
Epoch 3 - Accuracy: 3/52
Epoch 4 - Accuracy: 10/52
Epoch 5 - Accuracy: 11/52
Epoch 6 - Accuracy: 13/52
Epoch 7 - Accuracy: 17/52
Epoch 8 - Accuracy: 25/52
Epoch 9 - Accuracy: 33/52
Epoch 10 - Accuracy: 41/52
Epoch 11 - Accuracy: 36/52
Epoch 12 - Accuracy: 38/52
Epoch 13 - Accuracy: 36/52
Epoch 14 - Accuracy: 33/52
Epoch 15 - Accuracy: 42/52
Epoch 16 - Accuracy: 36/52
Epoch 17 - Accuracy: 26/52
Epoch 18 - Accuracy: 31/52
Epoch 19 - Accuracy: 31/52
Epoch 20 - Accuracy: 34/52
Epoch 21 - Accuracy: 47/52
Epoch 22 - Accuracy: 48/52
Epoch 23 - Accuracy: 50/52
Epoch 24 - Accuracy: 52/52
All items are correctly classified
number of epochs: 24
(26, 36)


array([[-1.59242459e-02, -5.72790780e-02,  1.92263587e-02,
         2.10161455e-02,  1.99533293e-02, -4.16712972e-02,
        -1.63030132e-03, -4.07090517e-03, -2.00250146e-03,
        -9.61054547e-03,  6.42920276e-03,  6.19976871e-04,
        -6.17557185e-03, -1.69790775e-02, -3.01960338e-03,
         1.99247519e-02,  9.21892418e-03,  1.00776481e-02,
         4.16763126e-04,  8.51827898e-03,  3.37431905e-02,
         1.27301839e-03,  2.66961195e-02,  1.23674460e-02,
         2.27300629e-02,  2.23755715e-02, -1.49070458e-03,
        -6.66997333e-03, -9.24788889e-03, -1.83548941e-02,
         2.54605765e-02,  1.76749007e-02, -3.86072566e-02,
        -5.00907052e-02, -3.86741880e-02,  3.41437252e-02],
       [-3.00356525e-02,  4.38014004e-03,  9.02985079e-03,
         7.61123382e-03,  9.69176756e-03, -4.26237742e-02,
        -1.60016668e-02, -4.00318384e-03, -2.25184550e-03,
        -9.13023545e-03, -3.17724465e-03, -1.40399136e-02,
        -2.23101972e-03, -1.72219326e-02, -3.09795801e-

# 8. Repeat experiment (6) with your softmax classifier. You can find the formula for applying softmax using NumPy in the documentation for SciPy’s softmax() function.

In [24]:
test_model(TRAINING_SET_2D, mul_log_reg_model)

Prediction: A, Target: A, True
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Prediction: G, Target: G, True
Prediction: H, Target: H, True
Prediction: I, Target: I, True
Prediction: J, Target: J, True
Prediction: K, Target: K, True
Prediction: L, Target: L, True
Prediction: M, Target: M, True
Prediction: N, Target: N, True
Prediction: O, Target: O, True
Prediction: P, Target: P, True
Prediction: Q, Target: Q, True
Prediction: R, Target: R, True
Prediction: S, Target: S, True
Prediction: T, Target: T, True
Prediction: U, Target: U, True
Prediction: V, Target: V, True
Prediction: W, Target: W, True
Prediction: X, Target: X, True
Prediction: Y, Target: Y, True
Prediction: Z, Target: Z, True
Prediction: A, Target: A, True
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Predicti

In [25]:
test_model(TEST_SET_2D, mul_log_reg_model)

Prediction: A, Target: A, True
Prediction: B, Target: B, True
Prediction: C, Target: C, True
Prediction: D, Target: D, True
Prediction: E, Target: E, True
Prediction: F, Target: F, True
Prediction: G, Target: G, True
Prediction: H, Target: H, True
Prediction: I, Target: I, True
Prediction: J, Target: J, True
Prediction: K, Target: K, True
Prediction: L, Target: L, True
Prediction: M, Target: M, True
Prediction: N, Target: N, True
Prediction: O, Target: O, True
Prediction: P, Target: P, True
Prediction: Q, Target: Q, True
Prediction: R, Target: R, True
Prediction: S, Target: S, True
Prediction: T, Target: T, True
Prediction: U, Target: U, True
Prediction: V, Target: V, True
Prediction: N, Target: W, False
Prediction: X, Target: X, True
Prediction: Y, Target: Y, True
Prediction: E, Target: Z, False

Accuracy: 24/26 Percentage: 92.31%


# 9. Use your trained classifiers to identify the letters in MESSAGE
## If you completed Project 1, compare the performance of that project with the performance with these classifiers. If you did not complete Project 1, use the sample code provided above under Libraries and Code.
## If you completed Project 2 compare the performance of that project with the performance of these classifiers.


In [26]:
def test_model_with_message(model):
    DECODED = 'WATCHJEOPARDYALEXTREBEKSFUNTVQUIZGAME'
    pred_msg = ''
    for letter in MESSAGE_2D:
        ascii_idx = np.argmax(np.dot(model, letter))
        y_pred = chr(ascii_idx + ord('A'))
        pred_msg += y_pred
    print(f'MSG : {DECODED}')
    print(f'Pred: {pred_msg}')
    correct = 0
    for true_letter, pred_letter in zip(DECODED, pred_msg):
        if true_letter == pred_letter:
            correct += 1
    print(f'accuracy: {correct/len(DECODED)*100:.2f}%')

In [27]:
# test with multiclass perceptron model
test_model_with_message(mul_pct_model)

MSG : WATCHJEOPARDYALEXTREBEKSFUNTVQUIZGAME
Pred: OGECWSEGFHPDTATEXJRGSEKCEVNTYJUTEGJME
accuracy: 40.54%


In [28]:
# test with multinomial logistic regresstion model
test_model_with_message(mul_log_reg_model)

MSG : WATCHJEOPARDYALEXTREBEKSFUNTVQUIZGAME
Pred: NATCHJFOFARDHALEXTRFBEKSFUNTVQUIEGAME
accuracy: 83.78%


### Based on our observations, the multiclass perceptron performs worse than the perceptron model in project 1. But, the multinomial logistic regression performs much better than single perceptron model in project 1 
- When testing against MESSAGE, the accuracy of single perceptron is around 43.24% in project 1

### Compare to project 2, both models do not perform as good as the Sequential Keras model does
- When testing against MESSAGE, the accuracy of the model is around 92.30% in project 2